RANDOM SEARCH TO FIND OPTIMAL HYPER-PARAMETERS

The main challenge lies in bridging scikit-learn's RandomizedSearchCV with PyTorch nn.Module.
- RandomizedSearchCV expects a scikit-learn estimator, which is a class that implements fit(), predict(), and potentially score().
- Interpolation(nn.Module) is a PyTorch model, not directly a scikit-learn estimator.

We'll do this with ` skorch ` where `NeuralNetRegressor`provides its own trainig loop. It also implements early stopping.

#### RANDOM SEARCH - Scikit Library

We'll be logging the best parameters and model in order to:
- Compare all runs
- Track randomness/reproducibility

In [ ]:
# Define the Search Space for the Randomly-searched parameters
# internal modules in skorch require the prefix module__
param_distributions = {
    'module__hidden_layer_sizes': [(256, 128), (128, 64), (128, 64, 32), (64, 128, 64), (64, 64, 64)],
    'module__activation': [nn.ReLU, nn.LeakyReLU, nn.ELU, nn.SiLU],
    'module__dropout': [0.1, 0.2, 0.4, 0.5],
    'lr': np.logspace(-4, -2, 20),
    'batch_size': [32, 64, 128, 256],
}

# Defined hyperparameters
input_size = 2  # baryon number and temperature
output_size = 2 # pressure and entropy

n_iterations = 40
epochs = 200
patience = 25

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Aditional metrics
def prediction_variance(y_true, y_pred):
    return np.var(y_pred)

r2_scorer = make_scorer(r2_score, greater_is_better=True)
var_scorer = make_scorer(prediction_variance, greater_is_better=True)

# Define the model with Skorch wrapper
model_estimator = NeuralNetRegressor(
    module=Interpolation,
    module__in_size=input_size,
    module__out_size=output_size,
    max_epochs=epochs,
    lr=0.01,  # Will be overridden during random search
    batch_size=64, #Likewise
    optimizer=torch.optim.Adam,
    criterion=nn.MSELoss,
    callbacks=[EarlyStopping(monitor='valid_loss', patience=patience)],
)

# Define the CrossValidation
cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=42)

RandomSearchCV implements Cross Validations KFold, RepeatedKFold is recommended for regression tasks

It also includes a scoring, the metric must be maximizing: better models result in larger scores
For regression, a negative error measure (‘neg_mean_absolute_error‘) makes values closer to zero to represent less prediction error by the model.

Once defined, the search is performed by calling the fit() function and providing a dataset used to train and evaluate model hyperparameter combinations using cross-validation.


In [ ]:
def save_random_search_results(search_obj, name_prefix="logP_logS_search"):
    base_dir = # path to save random search results
    exp_dir = os.path.join(base_dir, f"{name_prefix}")
    os.makedirs(exp_dir, exist_ok=True)

    # Create CSV for saving results of random search
    results_df = pd.DataFrame(search_obj.cv_results_)
    results_df.to_csv(os.path.join(exp_dir, "results_full.csv"), index=False)

    with open(os.path.join(exp_dir, "results_full.json"), "w") as f:
        json.dump((search_obj.cv_results_), f, indent=2)

    print(f"\n Saved results at: {exp_dir}")
    return exp_dir

In [ ]:
random_search = RandomizedSearchCV(
    estimator=model_estimator,
    param_distributions=param_distributions,
    n_iter=n_iterations,
    cv=cv,
    scoring={
        'mean_mse': 'neg_mean_squared_error',
        'r2': r2_scorer,
        'var_pred': var_scorer
    },
    refit='r2',  # R² as model score
    random_state=42,
    n_jobs=-1,
    verbose=2
)

# Convert combined train + validation data to NumPy for RandomizedSearchCV
x_train_val = np.concatenate((in_train_processed, in_val_processed), axis=0).astype(np.float32)
y_train_val = np.concatenate((out_train_processed, out_val_processed), axis=0).astype(np.float32)

# Calculate random search and log time
start_time = time.time()
random_search.fit(x_train_val, y_train_val)
end_time = time.time() - start_time
print(f"\n Randomized Search complete.")
print(f"Total time: {end_time} sec")

save_random_search_results(random_search)

In [ ]:
# Log results
logfile = # path where results_full.csv is saved
log_dir = os.path.dirname(logfile)
os.makedirs(log_dir, exist_ok=True)

results_df = pd.read_csv(logfile)

# Clean up and rename columns for clarity
columns_to_log = [
    'param_module__hidden_layer_sizes',
    'param_lr',
    'param_module__dropout',
    'param_module__activation',
    'param_batch_size',
    'rank_test_mean_mse',
    'rank_test_r2',
    'rank_test_var_pred',
    'mean_fit_time',
    'std_fit_time',
    'mean_test_mean_mse',
    'mean_test_r2',
    'mean_test_var_pred'
]

df_filtered = results_df[columns_to_log].copy()
df_filtered.rename(columns={
    'param_module__hidden_layer_sizes': 'hidden_layer_sizes',
    'param_lr': 'learning_rate',
    'param_module__dropout': 'dropout',
    'param_module__activation': 'activation_fn',
    'param_batch_size': 'batch_size',
    'mean_test_mean_mse': 'mean_neg_mse',
    'mean_test_r2': 'mean_r2',
    'mean_test_var_pred': 'mean_var'
}, inplace=True)

df_filtered['mean_mse'] = -df_filtered['mean_neg_mse'] # Convert to positive MSE

df_filtered.to_csv(logfile, index=False)
print(f"\nAll search results saved to: {logfile}")